In [1]:
import pandas as pd

In [3]:
df = pd.read_csv("/Users/bruce/Documents/Study/phishing ML/Supportive_Docs/my_final_dataset.csv")

In [4]:
df.isnull().sum()

url       0
label     0
result    0
dtype: int64

In [5]:
print(df['label'].value_counts())
print(df['result'].value_counts())

malicious    108523
benign        35378
Name: label, dtype: int64
1    108523
0     35378
Name: result, dtype: int64


In [6]:
from urllib.parse import urlparse

In [7]:
df_process = df

In [8]:
#1. get length of URL
df_process['url_length'] = df_process['url'].apply(lambda x: len(x))


# In[68]:


#2. get length of Hostname
df_process['hostname_length'] = df_process['url'].apply(lambda x: len(str(urlparse(x).hostname)))


# In[69]:


#3. get the length of the path from URL
df_process['path_length'] = df_process['url'].apply(lambda x: len(urlparse(x).path))


# In[70]:


#4. get the length of first directory
def fd(URL):
    a=urlparse(URL).path
    if str(a[0:2]) == '//':
        return len(a.split('/')[2])
    else:
        try:
            return len(a.split('/')[1])
        except:
            return 0

df_process['FirstDir_length'] = df_process['url'].apply(lambda v: fd(v))


# In[71]:


df_process.head()


# ### Counts of XYZ 

# In[72]:


#1. total -
df_process['total-'] = df_process['url'].apply(lambda i: i.count('-'))


# In[73]:


#2. total @
df_process['total@'] = df_process['url'].apply(lambda i: i.count('@'))


# In[74]:


#3. total ?
df_process['total?'] = df_process['url'].apply(lambda i: i.count('?'))


# In[75]:


#4. total %
df_process['total%'] = df_process['url'].apply(lambda i: i.count('%'))


# In[76]:


#5. total .
df_process['total.'] = df_process['url'].apply(lambda i: i.count('.'))


# In[77]:


#6. total =
df_process['total='] = df_process['url'].apply(lambda i: i.count('='))


# In[78]:


#7. total http
df_process['totalhttp'] = df_process['url'].apply(lambda i: i.count('http'))


# In[79]:


#8. total https
df_process['totalhttps'] = df_process['url'].apply(lambda i: i.count('https'))


# In[80]:


#9. total www
df_process['totalwww'] = df_process['url'].apply(lambda i: i.count('www'))


# In[81]:


#10 count total digits in the URL
def totaldigits(url):
    digits = 0
    for i in url:
        if i.isnumeric():
            digits = digits + 1
    return digits
df_process['totaldigits']= df_process['url'].apply(lambda i: totaldigits(i))


# In[82]:


#11 count total letters in the URL
def totalletters(url):
    letters = 0
    for i in url:
        if i.isalpha():
            letters = letters + 1
    return letters
df_process['totalletters']= df_process['url'].apply(lambda i: totalletters(i))


# In[83]:


#12 count total directories in the URL
def totaldirs(url):
    urldir = urlparse(url).path
    return urldir.count('/')
df_process['totaldirs'] = df_process['url'].apply(lambda i: totaldirs(i))


# In[84]:


#13 count of dots in netloc
df_process['totalnetlocdots'] = df_process['url'].apply(lambda i: urlparse(i).netloc.count('.'))


# In[85]:


# Internet Assigned Numbers Authority (IANA) approved list of TLD's
TLD_List = []
f=open("tld.txt","r")
for i in f:
    TLD_List.append(i.removesuffix('\n').lower())
print(TLD_List)


# In[86]:


#14 Count of TLD's in the netloc


def count_TLD(c):
    count=0

    for i in c:
        if i in TLD_List:
            count+=1
    return count

df_process['totalTLD'] = df_process['url'].apply(lambda i: count_TLD(urlparse(i).netloc.split('.')))


# ### Classification Features

# In[87]:


#1 Use of IP or not in domain

import re
def having_ip_address(url):
    match = re.search(
        '(([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\.'
        '([01]?\\d\\d?|2[0-4]\\d|25[0-5])\\/)|'  # IPv4
        '((0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\.(0x[0-9a-fA-F]{1,2})\\/)' # IPv4 in hexadecimal
        '(?:[a-fA-F0-9]{1,4}:){7}[a-fA-F0-9]{1,4}', url)  # Ipv6
    if match:
        # print match.group()
        return -1
    else:
        # print 'No matching pattern found'
        return 1
df_process['use_of_ip'] = df_process['url'].apply(lambda i: having_ip_address(i))


# In[88]:


#2 use of url shortening service
def shortening_service(url):
    match = re.search('bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|'
                      'yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|'
                      'short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|'
                      'doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|'
                      'db\.tt|qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|'
                      'q\.gs|is\.gd|po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|'
                      'x\.co|prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|'
                      'tr\.im|link\.zip\.net',
                      url)
    if match:
        return -1
    else:
        return 1
df_process['short_url'] = df_process['url'].apply(lambda i: shortening_service(i))


# In[89]:


df_process.head()


# In[90]:


df_process.shape


# In[91]:


df_process.info()


# In[92]:


df_process['label'].value_counts()


# In[93]:


df_process.to_csv("URL_test.csv")



['aaa', 'aarp', 'abarth', 'abb', 'abbott', 'abbvie', 'abc', 'able', 'abogado', 'abudhabi', 'ac', 'academy', 'accenture', 'accountant', 'accountants', 'aco', 'actor', 'ad', 'adac', 'ads', 'adult', 'ae', 'aeg', 'aero', 'aetna', 'af', 'afl', 'africa', 'ag', 'agakhan', 'agency', 'ai', 'aig', 'airbus', 'airforce', 'airtel', 'akdn', 'al', 'alfaromeo', 'alibaba', 'alipay', 'allfinanz', 'allstate', 'ally', 'alsace', 'alstom', 'am', 'amazon', 'americanexpress', 'americanfamily', 'amex', 'amfam', 'amica', 'amsterdam', 'analytics', 'android', 'anquan', 'anz', 'ao', 'aol', 'apartments', 'app', 'apple', 'aq', 'aquarelle', 'ar', 'arab', 'aramco', 'archi', 'army', 'arpa', 'art', 'arte', 'as', 'asda', 'asia', 'associates', 'at', 'athleta', 'attorney', 'au', 'auction', 'audi', 'audible', 'audio', 'auspost', 'author', 'auto', 'autos', 'avianca', 'aw', 'aws', 'ax', 'axa', 'az', 'azure', 'ba', 'baby', 'baidu', 'banamex', 'bananarepublic', 'band', 'bank', 'bar', 'barcelona', 'barclaycard', 'barclays', 'bar

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143901 entries, 0 to 143900
Data columns (total 23 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   url              143901 non-null  object
 1   label            143901 non-null  object
 2   result           143901 non-null  int64 
 3   url_length       143901 non-null  int64 
 4   hostname_length  143901 non-null  int64 
 5   path_length      143901 non-null  int64 
 6   FirstDir_length  143901 non-null  int64 
 7   total-           143901 non-null  int64 
 8   total@           143901 non-null  int64 
 9   total?           143901 non-null  int64 
 10  total%           143901 non-null  int64 
 11  total.           143901 non-null  int64 
 12  total=           143901 non-null  int64 
 13  totalhttp        143901 non-null  int64 
 14  totalhttps       143901 non-null  int64 
 15  totalwww         143901 non-null  int64 
 16  totaldigits      143901 non-null  int64 
 17  totallette

In [28]:
df_process.head()

,url,label,result,url_length,hostname_length,path_length,FirstDir_length,total-,total@,total?,...,totalhttp,totalhttps,totalwww,totaldigits,totalletters,totaldirs,totalnetlocdots,totalTLD,use_of_ip,short_url
0,http://1337x.to/torrent/1048648/American-Snipe...,benign,0,83,8,68,7,8,0,0,...,1,0,0,18,49,4,1,1,1,1
1,http://1337x.to/torrent/1110018/Blackhat-2015-...,benign,0,83,8,68,7,9,0,0,...,1,0,0,23,43,4,1,1,1,1
2,http://1337x.to/torrent/1122940/Blackhat-2015-...,benign,0,83,8,68,7,9,0,0,...,1,0,0,22,44,4,1,1,1,1
3,http://1337x.to/torrent/1124395/Fast-and-Furio...,benign,0,83,8,68,7,11,0,0,...,1,0,0,18,46,4,1,1,1,1
4,http://1337x.to/torrent/1145504/Avengers-Age-o...,benign,0,83,8,68,7,9,0,0,...,1,0,0,18,48,4,1,1,1,1


In [20]:
df_train= pd.read_csv("URL_phase1.csv")

In [21]:
df_train.head()

,Unnamed: 0,url,label,result,url_length,hostname_length,path_length,FirstDir_length,total-,total@,...,totalhttp,totalhttps,totalwww,totaldigits,totalletters,totaldirs,totalnetlocdots,totalTLD,use_of_ip,short_url
0,0,https://www.google.com,benign,0,22,14,0,0,0,0,...,1,1,1,0,17,0,2,2,1,1
1,1,https://www.youtube.com,benign,0,23,15,0,0,0,0,...,1,1,1,0,18,0,2,2,1,1
2,2,https://www.facebook.com,benign,0,24,16,0,0,0,0,...,1,1,1,0,19,0,2,1,1,1
3,3,https://www.baidu.com,benign,0,21,13,0,0,0,0,...,1,1,1,0,16,0,2,2,1,1
4,4,https://www.wikipedia.org,benign,0,25,17,0,0,0,0,...,1,1,1,0,20,0,2,1,1,1


In [22]:
# model training 

In [36]:
# finalize train dataset

In [23]:
df_train.head()

,Unnamed: 0,url,label,result,url_length,hostname_length,path_length,FirstDir_length,total-,total@,...,totalhttp,totalhttps,totalwww,totaldigits,totalletters,totaldirs,totalnetlocdots,totalTLD,use_of_ip,short_url
0,0,https://www.google.com,benign,0,22,14,0,0,0,0,...,1,1,1,0,17,0,2,2,1,1
1,1,https://www.youtube.com,benign,0,23,15,0,0,0,0,...,1,1,1,0,18,0,2,2,1,1
2,2,https://www.facebook.com,benign,0,24,16,0,0,0,0,...,1,1,1,0,19,0,2,1,1,1
3,3,https://www.baidu.com,benign,0,21,13,0,0,0,0,...,1,1,1,0,16,0,2,2,1,1
4,4,https://www.wikipedia.org,benign,0,25,17,0,0,0,0,...,1,1,1,0,20,0,2,1,1,1


In [24]:
df_ml_train = df_train

In [25]:
df_ml_train.drop(['Unnamed: 0','url','label'], axis=1, inplace =True)

In [27]:
df_ml_train.head()

,result,url_length,hostname_length,path_length,FirstDir_length,total-,total@,total?,total%,total.,...,totalhttp,totalhttps,totalwww,totaldigits,totalletters,totaldirs,totalnetlocdots,totalTLD,use_of_ip,short_url
0,0,22,14,0,0,0,0,0,0,2,...,1,1,1,0,17,0,2,2,1,1
1,0,23,15,0,0,0,0,0,0,2,...,1,1,1,0,18,0,2,2,1,1
2,0,24,16,0,0,0,0,0,0,2,...,1,1,1,0,19,0,2,1,1,1
3,0,21,13,0,0,0,0,0,0,2,...,1,1,1,0,16,0,2,2,1,1
4,0,25,17,0,0,0,0,0,0,2,...,1,1,1,0,20,0,2,1,1,1


In [30]:
df_ml_train_X = df_ml_train.loc[:, df_ml_train.columns != 'result']

In [31]:
df_ml_train_X.head()

,url_length,hostname_length,path_length,FirstDir_length,total-,total@,total?,total%,total.,total=,totalhttp,totalhttps,totalwww,totaldigits,totalletters,totaldirs,totalnetlocdots,totalTLD,use_of_ip,short_url
0,22,14,0,0,0,0,0,0,2,0,1,1,1,0,17,0,2,2,1,1
1,23,15,0,0,0,0,0,0,2,0,1,1,1,0,18,0,2,2,1,1
2,24,16,0,0,0,0,0,0,2,0,1,1,1,0,19,0,2,1,1,1
3,21,13,0,0,0,0,0,0,2,0,1,1,1,0,16,0,2,2,1,1
4,25,17,0,0,0,0,0,0,2,0,1,1,1,0,20,0,2,1,1,1


In [32]:
df_ml_train_Y = df_ml_train['result']

In [33]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()

In [34]:
x_sample_train, y_sample_train = SMOTE().fit_resample(df_ml_train_X, df_ml_train_Y.values.ravel())

In [35]:
print(df_ml_train_X.shape)
print(df_ml_train_X.shape)
print(x_sample_train.shape)
print(y_sample_train.shape)

(450176, 20)
(450176, 20)
(691476, 20)
(691476,)


In [37]:
#finalize test dataset

In [38]:
df_test= pd.read_csv("URL_test.csv")

In [39]:
df_test.head()

,Unnamed: 0,url,label,result,url_length,hostname_length,path_length,FirstDir_length,total-,total@,...,totalhttp,totalhttps,totalwww,totaldigits,totalletters,totaldirs,totalnetlocdots,totalTLD,use_of_ip,short_url
0,0,http://1337x.to/torrent/1048648/American-Snipe...,benign,0,83,8,68,7,8,0,...,1,0,0,18,49,4,1,1,1,1
1,1,http://1337x.to/torrent/1110018/Blackhat-2015-...,benign,0,83,8,68,7,9,0,...,1,0,0,23,43,4,1,1,1,1
2,2,http://1337x.to/torrent/1122940/Blackhat-2015-...,benign,0,83,8,68,7,9,0,...,1,0,0,22,44,4,1,1,1,1
3,3,http://1337x.to/torrent/1124395/Fast-and-Furio...,benign,0,83,8,68,7,11,0,...,1,0,0,18,46,4,1,1,1,1
4,4,http://1337x.to/torrent/1145504/Avengers-Age-o...,benign,0,83,8,68,7,9,0,...,1,0,0,18,48,4,1,1,1,1


In [41]:
df_test.drop(['Unnamed: 0','url','label'], axis=1, inplace =True)

In [42]:
df_test_X = df_test.loc[:, df_test.columns != 'result']

In [43]:
df_test_Y = df_test['result']

In [44]:
print(df_test_X.shape)
print(df_test_Y.shape)

(143901, 20)
(143901,)


### KNN with Hyper-parameter tuning

In [49]:
from sklearn.neighbors import KNeighborsClassifier

In [52]:
knn_model_hyper = KNeighborsClassifier(n_neighbors=10,weights='distance',algorithm='ball_tree',
                                       leaf_size=30, p=1, metric='manhattan')
                                       

In [53]:
knn_model_hyper.fit(x_sample_train,y_sample_train)

KNeighborsClassifier(algorithm='ball_tree', metric='manhattan', n_neighbors=10,
                     p=1, weights='distance')

In [54]:
knn_pred = knn_model_hyper.predict(df_test_X)

In [65]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [56]:
knn_accuracy = accuracy_score_score(df_test_Y, knn_pred)

In [57]:
print("Accuracy of KNN (Hyper-parameter) on Testing dataset : ",round(knn_accuracy,3))

Accuracy of KNN (Hyper-parameter) on Testing dataset :  0.444


### KNN : without hyper-parameter tuning

In [59]:
knn_model_2 = KNeighborsClassifier()

In [60]:
knn_model_2.fit(x_sample_train,y_sample_train)

KNeighborsClassifier()

In [61]:
knn_pred_2 = knn_model_2.predict(df_test_X)

In [67]:
knn_accuracy_2 = accuracy_score(df_test_Y, knn_pred_2)

In [68]:
print("Accuracy of KNN (without Hyper-parameters) on Testing dataset : ",round(knn_accuracy_2,3))

Accuracy of KNN (without Hyper-parameters) on Testing dataset :  0.457


### Decision Tree without Hyper-parameter tuning

In [69]:
from sklearn.tree import DecisionTreeClassifier

In [70]:
dt_model = DecisionTreeClassifier()

In [71]:
dt_model.fit(x_sample_train, y_sample_train)

DecisionTreeClassifier()

In [72]:
dt_model_pred = dt_model.predict(df_test_X)

In [73]:
dt_accuracy = accuracy_score(df_test_Y, dt_model_pred)

In [74]:
print('The accuracy of Decision Tree Classifier is : ', round(dt_accuracy,3))

The accuracy of Decision Tree Classifier is :  0.753


### Decision Tree with Hyper-parameter tuning

In [75]:
dt_model_hyper = DecisionTreeClassifier(criterion='entropy')

In [76]:
dt_model_hyper.fit(x_sample_train, y_sample_train)

DecisionTreeClassifier(criterion='entropy')

In [77]:
dt_model_pred_hyper = dt_model_hyper.predict(df_test_X)

In [78]:
dt_accuracy_hyper = accuracy_score(df_test_Y, dt_model_pred_hyper)

In [79]:
print('The accuracy of Decision Tree Classifier (with hyper-parameter tuning) is : ', round(dt_accuracy_hyper,3))

The accuracy of Decision Tree Classifier (with hyper-parameter tuning) is :  0.753


### Random Forest without Hyper-parameters 

In [87]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier()

rf_model.fit(x_sample_train, y_sample_train)

rf_model_pred = rf_model.predict(df_test_X)

rf_accuracy = accuracy_score(df_test_Y, rf_model_pred)

print('The accuracy of Random Forest(without hyper-parameer) is : ' , round(rf_accuracy,3))

The accuracy of Random Forest(without hyper-parameer) is :  0.754


### Random Forest with Hyper-parameters 

In [88]:
from sklearn.ensemble import RandomForestClassifier

rf_model_hyper = RandomForestClassifier(n_estimators=150,max_features=None, criterion='entropy' )

rf_model_hyper.fit(x_sample_train, y_sample_train)

rf_model_hyper_pred = rf_model_hyper.predict(df_test_X)

rf_hyper_accuracy = accuracy_score(df_test_Y, rf_model_hyper_pred)

print('The accuracy of Random Forest(with hyper-parameer tuning) is : ' , round(rf_hyper_accuracy,3))

The accuracy of Random Forest(with hyper-parameer tuning) is :  0.754
